In [37]:
import configparser
import glob
import os
import pandas as pd
import subprocess
import sys
import tarfile 
from urllib.request import urlretrieve
import json
import tempfile
import tensorflow as tf

In [38]:
sys.path.append("../src")
from utils import str_to_value

In [39]:
sys.path.append("../bert")
import modeling

In [ ]:
CURDIR = os.getcwd()
CONFIGPATH = os.path.join(CURDIR, os.pardir, 'config.ini')
config = configparser.ConfigParser()
config.read(CONFIGPATH)

In [41]:
FILEURL = config['FINETUNING-DATA']['FILEURL']
FILEPATH = config['FINETUNING-DATA']['FILEPATH']
EXTRACTDIR = config['FINETUNING-DATA']['TEXTDIR']

In [42]:
PRETRAINED_MODEL_PATH = '../model/bert-wiki-ja/model.ckpt-1400000'

In [43]:
bert_config_file = tempfile.NamedTemporaryFile(mode='w+t', encoding='utf-8', suffix='.json')
bert_config_file.write(json.dumps({k:str_to_value(v) for k,v in config['BERT-CONFIG'].items()}))
bert_config_file.seek(0)
bert_config_file_path = str(bert_config_file.name)
bert_config = modeling.BertConfig.from_json_file(bert_config_file.name)

In [44]:
!echo 'すべての人間は、生れながらにして自由であり、かつ、尊厳と権利とについて平等である。 ||| 人間は、理性と良心とを授けられており、互いに同胞の精神をもって行動しなければならない。' > /tmp/input.txt

In [ ]:
!python ../src/extract_features.py \
  --input_file=/tmp/input.txt \
  --output_file=output.json \
  --vocab_file=../model/bert-wiki-ja/wiki-ja.vocab \
  --model_file=../model/bert-wiki-ja/wiki-ja.model \
  --bert_config_file={bert_config_file.name} \
  --init_checkpoint={PRETRAINED_MODEL_PATH} \
  --layers=-1,-2,-3,-4 \
  --max_seq_length=128 \
  --batch_size=8

Let's have a look at the features of the last layer for the word "人間".

The 0-th token is always [CLS], and the 1st token of a sentence is [▁]. So the word comes in 3rd position.

In [59]:
import json

outputs = json.load(open('output.json'))
print(outputs['features'][3]['token'])

人間


The last year is layer 0, the one before is layer -1, etc...
The embeddings are stored in the *values* entry.

In [66]:
embeddings = outputs['features'][3]['layers'][0]['values']
print(len(embeddings))

768
